# Příjem obrázku a odeslání do Edge Impulse

In [4]:
import serial
import time
import os
import subprocess
import tempfile
from PIL import Image
from io import BytesIO

# ⚙️ Nastavení
usbPort = '/dev/tty.usbmodem2101'  # Mac/Linux
# usbPort = 'COM4'  # Windows
baudRate = 115200
EDGE_IMPULSE_CLI = 'edge-impulse-uploader'
LABEL = 'kostka'  # štítek pro dataset

# Připojení k sériovému portu
try:
    ser = serial.Serial(usbPort, baudRate, timeout=5)
    print(f"Připojeno k {usbPort}")
except serial.SerialException:
    print(f"Nelze otevřít {usbPort}. Zkontroluj připojení ESP32-S3!")
    exit()

# Hlavní smyčka
while True:
    input("🖼Stiskni ENTER pro pořízení snímku... (nebo Ctrl+C pro ukončení)")

    ser.write(b'snap\n')
    print("Posílám příkaz 'snap'...")

    buffer = b""
    capturing = False

    while True:
        line = ser.readline()
        if b"BEGIN_IMAGE" in line:
            print("Začínám příjem obrázku...")
            buffer = b""
            capturing = True
        elif b"END_IMAGE" in line and capturing:
            print("Příjem dokončen. Zpracovávám...")

            try:
                image = Image.open(BytesIO(buffer))
            except Exception as e:
                print(f"Chyba při dekódování obrázku: {e}")
                break

            width, height = image.size
            print(f"Rozměry: {width}x{height}")

            # Oříznutí na čtverec
            if width != height:
                side = min(width, height)
                left = (width - side) // 2
                top = (height - side) // 2
                right = left + side
                bottom = top + side
                image = image.crop((left, top, right, bottom))
                print(f"Ořezáno na čtverec: {image.size}")

            # Dočasné uložení
            with tempfile.NamedTemporaryFile(suffix='.jpg', delete=False) as tmp:
                image.save(tmp.name, format='JPEG')
                temp_filename = tmp.name
                print(f"Dočasně uloženo do: {temp_filename}")

            # Odeslání do Edge Impulse
            print("Odesílám do Edge Impulse...")
            result = subprocess.run([
                EDGE_IMPULSE_CLI,
                '--label', LABEL,
                temp_filename
            ], capture_output=True, text=True)

            if result.returncode == 0:
                print("Snímek úspěšně odeslán.")
            else:
                print("Chyba při odesílání:")
                print(result.stderr)

            os.remove(temp_filename)  # smažeme dočasný soubor
            break  # hotovo
        elif capturing:
            buffer += line


Připojeno k /dev/tty.usbmodem2101
Posílám příkaz 'snap'...
Začínám příjem obrázku...
Příjem dokončen. Zpracovávám...
Rozměry: 640x480
Ořezáno na čtverec: (480, 480)
Dočasně uloženo do: /var/folders/6f/m_ls96295sl5v0q5fc8hxjx40000gn/T/tmpziw__ncc.jpg
Odesílám do Edge Impulse...
Snímek úspěšně odeslán.
Posílám příkaz 'snap'...
Začínám příjem obrázku...
Příjem dokončen. Zpracovávám...
Rozměry: 640x480
Ořezáno na čtverec: (480, 480)
Dočasně uloženo do: /var/folders/6f/m_ls96295sl5v0q5fc8hxjx40000gn/T/tmpcwbftyhk.jpg
Odesílám do Edge Impulse...
Snímek úspěšně odeslán.
Posílám příkaz 'snap'...
Začínám příjem obrázku...
Příjem dokončen. Zpracovávám...
Rozměry: 640x480
Ořezáno na čtverec: (480, 480)
Dočasně uloženo do: /var/folders/6f/m_ls96295sl5v0q5fc8hxjx40000gn/T/tmphuzj6qow.jpg
Odesílám do Edge Impulse...
Snímek úspěšně odeslán.


KeyboardInterrupt: Interrupted by user